In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import pandas as pd
from bs4 import BeautifulSoup

import os
chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver

driver = webdriver.Chrome(chromedriver)
driver.get('https://gi9641r1.cachefly.net/reviewsarchive/review-archive-legacy-pantheon14.html')

# Alternatives to Chrome: Firefox, phantomjs

In [2]:
# Selenium hands the page to Beautiful soup
driver.get('https://gi9641r1.cachefly.net/reviewsarchive/review-archive-legacy-pantheon14.html')
soup_level1=BeautifulSoup(driver.page_source, 'lxml')

In [3]:
#import requests
#import pandas as pd
#from bs4 import BeautifulSoup

class HTMLTableParser:
   
    def parse_url(self, url):
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'lxml')
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  
    
    def parse_soup(self, soup):
        return [(table['id'],self.parse_html_table(table))\
                for table in soup.find_all('table')]  

    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []

        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
            
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                    
            # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())

        # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")

        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                          index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                
        # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
        
        return df

In [5]:
hp = HTMLTableParser()
steam_users_table = hp.parse_soup(soup_level1)[0][1] # Grabbing the table from the tuple
steam_users_table

,Title,Issue,Platform,Score
0,,,,
1,Persona 5,2017-05,PS3,9.25
2,WWE 2K17,2016-12,PS3,5.75
3,WWE 2K17,2016-12,360,5.75
4,Skylanders Imaginators,2016-12,PS3,7
5,Skylanders Imaginators,2016-12,360,7
6,King's Quest: Chapter 4 – Snow Place Like Home,2016-12,PS3,6.5
7,King's Quest: Chapter 4 – Snow Place Like Home,2016-12,360,6.5
8,Pro Evolution Soccer 2017,2016-11,PS3,9.25
9,Pro Evolution Soccer 2017,2016-11,360,9.25
